# Chronos

In [117]:
import warnings
import transformers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from chronos import ChronosPipeline
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, root_mean_squared_error
from darts import TimeSeries
import plotly.graph_objs as go
import plotly.io as pio

In [118]:
# Import the data
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [119]:
train_df = load_and_prepare_data('../../data/Final_data/train_df_no_lags.csv')
test_df = load_and_prepare_data('../../data/Final_data/test_df_no_lags.csv')

train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [120]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data_no_lags.csv')

# Reset the index
df = df.reset_index()

target_column = "Day_ahead_price (€/MWh)"

In [121]:
# date of first forecast
start_date = "2022-07-01"

# date of last forecast
end_date = "2024-07-28"

In [122]:
# create a TimeSeries object from df
data = df[['Date', target_column]]
# transform to a pandas series
data = data.set_index('Date')
data = data.squeeze()
data

Date
2012-01-08    26.83
2012-01-09    47.91
2012-01-10    45.77
2012-01-11    47.83
2012-01-12    43.10
              ...  
2024-07-24    66.61
2024-07-25    78.34
2024-07-26    93.04
2024-07-27    80.74
2024-07-28    43.96
Name: Day_ahead_price (€/MWh), Length: 4586, dtype: float64

In [123]:
data.shape

(4586,)

## Chronos Pipeline

In [ ]:
# Define the size of the Chronos model
SIZE = "small#"

In [125]:
chronos_model = ChronosPipeline.from_pretrained(
    f"amazon/chronos-t5-{SIZE}",
    device_map="mps",
    torch_dtype=torch.bfloat16,
)

In [126]:
# save the start time
start_time = datetime.now()

# create a list for storing the forecasts
chronos_forecasts = []

transformers.set_seed(42)

NUM_SAMPLES = 100

# loop across the dates
for t in tqdm(range(data.index.get_loc(start_date), data.index.get_loc(end_date) + 1)):

    # extract the context window
    context = data.iloc[:t]

    # generate the one-step-ahead forecast
    chronos_forecast = chronos_model.predict(
        context=torch.from_numpy(context.values),
        prediction_length=1,
        num_samples=NUM_SAMPLES
    ).detach().cpu().numpy().flatten()

    # save the forecasts
    chronos_forecasts.append({
        "date": data.index[t],
        "actual": data.values[t],
        "mean": np.mean(chronos_forecast),
        "std": np.std(chronos_forecast, ddof=1),
    })

# cast the forecasts to data frame
chronos_forecasts = pd.DataFrame(chronos_forecasts)

# save the end time
end_time = datetime.now()

print(f"\nRunning time of Chronos model: {end_time - start_time}")

  0%|          | 0/759 [00:00<?, ?it/s]

100%|██████████| 759/759 [05:12<00:00,  2.43it/s]


Running time of Chronos model: 0:05:12.843571


In [127]:
chronos_forecasts.head()

,date,actual,mean,std
0,2022-07-01,314.38,302.450213,12.360555
1,2022-07-02,218.92,281.430225,13.408668
2,2022-07-03,200.11,202.544789,15.578612
3,2022-07-04,293.89,238.105888,13.341606
4,2022-07-05,318.37,306.686949,14.797759


In [128]:
chronos_forecasts.tail()

,date,actual,mean,std
754,2024-07-24,66.61,88.266341,8.058641
755,2024-07-25,78.34,81.586316,8.723337
756,2024-07-26,93.04,79.412249,8.255500
757,2024-07-27,80.74,74.211680,7.982154
758,2024-07-28,43.96,71.400777,8.611977


In [129]:
# Plot the result

# Create traces for actual and predicted values
trace_actual = go.Scatter(
    x=chronos_forecasts["date"].values,
    y=chronos_forecasts["actual"].values,
    mode='lines',
    name='Actual',
    line=dict(color='#3f4751', width=1)
)

trace_predicted = go.Scatter(
    x=chronos_forecasts["date"].values,
    y=chronos_forecasts["mean"].values,
    mode='lines',
    name='Predicted',
    line=dict(color='#009ad3', width=1)
)

# Create traces for confidence intervals
trace_std_1 = go.Scatter(
    x=chronos_forecasts["date"].values,
    y=chronos_forecasts["mean"].values + chronos_forecasts["std"].values,
    mode='lines',
    name='Predicted +/- 1 Std. Dev.',
    line=dict(color='#009ad3', width=0),
    fill='tonexty',
    fillcolor='rgba(0, 154, 211, 0.2)'
)

trace_std_1_neg = go.Scatter(
    x=chronos_forecasts["date"].values,
    y=chronos_forecasts["mean"].values - chronos_forecasts["std"].values,
    mode='lines',
    line=dict(color='#009ad3', width=0),
    showlegend=False,
    fill='tonexty',
    fillcolor='rgba(0, 154, 211, 0.2)'
)

trace_std_2 = go.Scatter(
    x=chronos_forecasts["date"].values,
    y=chronos_forecasts["mean"].values + 2 * chronos_forecasts["std"].values,
    mode='lines',
    name='Predicted +/- 2 Std. Dev.',
    line=dict(color='#009ad3', width=0),
    fill='tonexty',
    fillcolor='rgba(0, 154, 211, 0.1)'
)

trace_std_2_neg = go.Scatter(
    x=chronos_forecasts["date"].values,
    y=chronos_forecasts["mean"].values - 2 * chronos_forecasts["std"].values,
    mode='lines',
    line=dict(color='#009ad3', width=0),
    showlegend=False,
    fill='tonexty',
    fillcolor='rgba(0, 154, 211, 0.1)'
)

# Create the figure with all the traces
fig = go.Figure()
fig.add_trace(trace_actual)
fig.add_trace(trace_predicted)
fig.add_trace(trace_std_2)
fig.add_trace(trace_std_2_neg)
fig.add_trace(trace_std_1)
fig.add_trace(trace_std_1_neg)

# Set layout options
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Value',
    legend=dict(x=1.05, y=1),
    margin=dict(t=20, b=20, l=20, r=20),
    width=1200,
    height=450
)

# Show the plot
pio.show(fig)

In [130]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.metrics import rmse, mae, mape, smape, mse

# Convert the chronos_forecasts DataFrame to Darts TimeSeries objects
actual_series = TimeSeries.from_series(chronos_forecasts["actual"])
predicted_series = TimeSeries.from_series(chronos_forecasts["mean"])

# Calculate the metrics using Darts functions
rmse_value = rmse(actual_series, predicted_series)
mae_value = mae(actual_series, predicted_series)
mape_value = mape(actual_series, predicted_series)
smape_value = smape(actual_series, predicted_series)
mse_value = mse(actual_series, predicted_series)

# Create the metrics DataFrame
chronos_metrics = pd.DataFrame(
    columns=["Metric", "Value"],
    data=[
        {"Metric": "RMSE", "Value": rmse_value},
        {"Metric": "MAE", "Value": mae_value},
        {"Metric": "MAPE", "Value": mape_value},
        {"Metric": "SMAPE", "Value": smape_value},
        {"Metric": "MSE", "Value": mse_value},
    ]
).set_index("Metric")

In [131]:
chronos_metrics

,Value
Metric,
RMSE,36.305899
MAE,24.147636
MAPE,87.879958
SMAPE,24.092043
MSE,1318.118299


In [132]:
# Save Chronos metrics as csv
chronos_metrics.to_csv(f'chronos_metrics_{SIZE}.csv')

In [133]:
# Select only the 'date' and 'mean' columns and create a copy to avoid the warning
date_mean_data = chronos_forecasts[['date', 'mean']].copy()

# Rename 'mean' to 'Day_ahead_price'
date_mean_data.rename(columns={'mean': 'Day_ahead_price'}, inplace=True)

# Save to CSV
date_mean_data.to_csv(f'chronos_forecast_{SIZE}.csv', index=False)
print(f"Data saved to chronos_forecast_{SIZE}.csv")

Data saved to chronos_forecast_small.csv
